# Homework 5: Bayesian Graphical Models

- Name: Congxin (David) Xu
- Computing ID: cx2rx

### Honor Pledge: 
I have neither given nor received aid on this assignment.

### Problem 1
(30) This problem explores the use of variational approximation in Latent
Dirichlet Allocation (LDA). We will use the implementation in sklearn of
the variational approximation algorithm in [1].

#### Part a

Use the notation in the diagram in Figure 1 [2] to write the target
posterior distribution of the latent variables and parameters for the
general LDA method. Why do we use variational approximation
rather than conjugate priors or sampling to obtain this posterior
distribution?

**Answer**

The posterior distribution of the latent variables is 
$$ p(\theta, z | w, \alpha, \beta) = \frac{p(\theta, z, w| \alpha, \beta)}{p(w | \alpha, \beta)}  $$

The reason to use variational approximation rather than conjugate priors or sampling to obtain this posterior distribution is becuase for many models, it is intractable to compute the posterior distribution. Sampling methods are limited to smaller data problems but variational approximation can handle large problems.

#### Part b

Accident reports provide a good use-case for LDA since the narrative
information in these reports is frequently overlooked in safety analysis. LDA allows us to capture elements (topics) in this narrative
data and use them to better understand unsafe conditions. For this
use-case, modify the LDA class for Wikipedia in the `LDA Examples Wikipedia and Trains` jupyter notebook to perform LDA on
the accident narratives. About 10 years of these narratives are in
the json file, `TrainNarratives.txt`. Use this class to obtain 10
topics from the accident narratives.

In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import wikipedia
import nltk
from nltk.corpus import stopwords
import json

# Set stop words
stopWords = set(stopwords.words('english'))

In [2]:
class LDA_trains:
    """Creates a class for Latent Dirichlet Allocation using summaries from Wikipedia
    Input:
        reports = list of narratives from accident reports
        N_topics = number of topics for LDA to produce
        N_words = the number of words to show in a topic
        new_report = narrative for a new accident report not in the training set
    Methods:
        Topics = output the list of topics in the selected narratives
        Predict_Topics = Show the predicted probabilities for topics for a new accident narrative
            Input: new narrative
            """
    def __init__(self, reports, N_topics=3, N_words = 10):
        # the narrative reports
        self.reports = reports
        # initialize variables
        self.N_topics = N_topics
        self.N_words = N_words
        
        # Get the word counts in the reports
        self.countVectorizer = CountVectorizer(stop_words='english')
        self.termFrequency = self.countVectorizer.fit_transform(self.reports)
        self.Words = self.countVectorizer.get_feature_names()
        
    def Topics(self):
                
        # Obtain the estimates for the LDA model 
        self.lda = LatentDirichletAllocation(n_components=self.N_topics)
        self.lda.fit(self.termFrequency)
        
        # Obtain the list of the top N_words in the topics
        topics = list()
        for topic in self.lda.components_:
            topics.append([self.Words[i] for i in topic.argsort()[:-self.N_words - 1:-1]])
            
        # For each of the topics in the model add the top N_words the list of topics
        ### Your code here
        # Create column names for the output matrix
        cols = list()
        for i in range(self.N_words):
            cols.append("Word " + (str(i)))
            
        # Create a dataframe with the topic no. and the words in each topic 
        # output this dataframe 
        Topics_df = pd.DataFrame(topics, columns = cols)
        Topics_df.index.name = "Topics"
        return Topics_df
    
    def Predict_Topics(self, new_reports):
        self.new_reports = new_reports
        
        # Get the list of new accident report narratives
        # and the number of new narratives
        N_new_reports = len(self.new_reports)
        
        
        # For each of the new narratives 
        # obtain the estimated probabilities for each of the topics
        # in each of the new narratives as estimated by the LDA results
        # on the training set 
        new_report_topics = list()
        ### Your code here        
        for i in self.new_reports:
            new_report_topics.append(self.lda.\
                                     transform(self.countVectorizer.\
                                               transform([i])))
        
        # Recast the list of probabilities for topics as an array 
        # of size no. of new reports X no. of topics
        new_report_topics = np.array(new_report_topics).\
            reshape(N_new_reports, self.N_topics)
        
        # Create column names for the output dataframe
        cols = list()
        ### Your code here        
        for i in range(self.N_topics):
            cols.append("Topic "+(str(i)))
            
        # Create the dataframe whose rows contain topic probabilities for 
        # specificed narratives/reports
        ### Your code here
        New_Reports_df = pd.DataFrame(new_report_topics, columns = cols)        
        New_Reports_df.insert(0, 'Reports', self.new_reports)
        
        return New_Reports_df
                

In [3]:
# Train accident narratives are in a json file
# Read the JSON file with the narratives and convert to a list for the LDA analysis

with open('TrainNarratives.txt') as json_file:  
    Narrative_dict = json.load(json_file)
    
train_reports = list(Narrative_dict.values())
    
train_reports[0:3]

['UNITS 231-281(BACK TO BACK)  WERE COMING INTO UP DEISEL SHOP  WHEN THE LEFT WHEEL OF 281 RODE OVER RECENTLY REPAIRED SWITCH PLATE AND DERAILED. THE CAUSE WAS DETERMINED TO BE THE TRACK TELEMETRY IN THAT IT WAS TOO SHARP OF A CURVE.',
 'ENGINE 286 CAUGHT FIRE AT THE SPRINGFIELD, MA STATION DUE TO BEARINGS IN MAIN GENERATOR LET GO.',
 'TRAIN NO.#4 WITH ENGS 83/11/90/44 AND 11 CARS DERAILED 2 DEADHEAD CARS, C/44834 AND C/9639, WHILE MAKING A SHOVING MOVE ONTO TRACK 28.  THE DERAILMENT WAS DUE TO HIGH BUFF FORCES CAUSED JACKKNIFING OFDEADHEADING AMFLEET CAR 44834 LOCATED DIRECTLY BEHIND ENGINES DUE TO EXCESSIVE AMPERAGE GENERATED BY FOUR P42 LOCOMOTIVES SHOVING TRAIN AGAINST AN APPROXIMATELY 15-POUND BRAKE REDUCTION.']

In [4]:
lda_train = LDA_trains(reports = train_reports, N_topics = 10, N_words = 10)
lda_train.Topics()

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9
Topics,,,,,,,,,,
0,fuel,gallons,released,cars,diesel,spilled,gal,tank,lead,bnsf
1,cars,track,rail,derailed,car,switch,crew,derail,causing,train
2,car,derailed,track,cars,causing,end,switch,lead,shoving,derail
3,ns,derailed,loads,tons,empties,pulling,units,wheels,cars,train
4,car,cars,track,cut,bowl,humping,hump,rolled,humped,damage
5,train,emergency,derailed,crew,went,mp,cars,conductor,traveling,stopped
6,train,engineer,car,cars,conductor,stop,brake,derailment,went,slack
7,derailed,cars,track,rail,pulling,train,broken,hazardous,main,materials
8,track,cars,switch,crew,yard,lead,end,derailed,cut,lined


#### Part c

Use the class you developed for Problem 1b to obtain the probabilities
for each of the topics in the first 10 narratives in the `TrainNarratives.txt`
data set. What is the notation in Figure1 that represents these prob-
abilities?

**Answer**

The notation in Figure 1 that represents these probabilities is $\theta$.

In [5]:
lda_train.Predict_Topics(train_reports[:10])

,Reports,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,Topic 8,Topic 9
0,UNITS 231-281(BACK TO BACK) WERE COMING INTO ...,0.004546,0.004547,0.959082,0.004546,0.004546,0.004546,0.004548,0.004547,0.004546,0.004547
1,"ENGINE 286 CAUGHT FIRE AT THE SPRINGFIELD, MA ...",0.009091,0.009092,0.009093,0.009093,0.208054,0.009093,0.009092,0.009094,0.009092,0.719205
2,TRAIN NO.#4 WITH ENGS 83/11/90/44 AND 11 CARS ...,0.002326,0.098332,0.588089,0.002326,0.002326,0.002326,0.297296,0.002326,0.002326,0.002326
3,WHILE SHOVING TRAIN 624 SOUTH ON #30 TRACK AT ...,0.006250,0.006251,0.006251,0.006251,0.006251,0.006252,0.006252,0.006252,0.784815,0.165175
4,TRAIN 786 WAS STRUCK BY A FALLING TREE SOUTH O...,0.010001,0.010002,0.010002,0.010001,0.010001,0.909986,0.010000,0.010003,0.010001,0.010003
5,ENGINE 4403 OF NJT TRAIN 3204 HAD 90% OLD BREA...,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,0.005000,0.954998
6,AGR CREW DELIVERED CARS TO BNSF AT BNSF YARD A...,0.002273,0.002273,0.247247,0.002273,0.262689,0.002273,0.002273,0.002273,0.347539,0.128886
7,TRAIN #263 CAME INTO RUTHLAND YARD AND THEY WE...,0.002083,0.821133,0.002084,0.002084,0.002084,0.114560,0.049722,0.002084,0.002084,0.002084
8,CREW WAS SHOVING A CUT OF CARS EASTWARD TOWARD...,0.003847,0.273505,0.003847,0.003847,0.003847,0.003847,0.003847,0.003847,0.695719,0.003847
9,"WHILE BUILDING TAIN, 1130 TRIMMER DERAILED FOU...",0.251954,0.007694,0.551449,0.007694,0.007695,0.007693,0.007693,0.142741,0.007694,0.007693


#### Part d

Briefly explain how a safety engineer at Federal Railroad Administration could use the results you obtain in Problem 1c to improve safety for trains.

**Answer**

The safety engineer at Federal Rialroad Administration could look at the train report and the topic that report belongs to. Then, the safety engineer will be able to review the words under that topic and pay special attention to those areas when he/she conduct the safety checks.

### Problem 2
(10) Neural network are a graphical model, Markov Networks, that can
be analyzed with Bayesian methods and Boltzmann machines are examples. Provide an explanation for the intractability of the calculation of the
partition function, Z, in a Boltzmann machine. Explain how restricted Boltzmann machines help with this problem. Also explain how MCMC
and variational methods can provide approximations to Z in restricted
Boltzman machines

**Answer**

In an unrestricted Boltzmann machines, the hidden units are connected and the observed units are also connected. We defined the partition function, $Z$, as the sum of products of the maximal cliques. The computation of the potential function at each maximal cliques will grow exponentially because the hidden units and the observed units are connected with each other.  

By removing the connections among the hidden units and the connections among observed units, the number of maximal cliques will decrease dramatically. Now, the fully connected units will only be pairs, one hidden node to one observed node. In the restricted Bltzmann machine, the number of maximal cliques will be the number of hidden units times the number of observed units. 

We can use MCMC to do a samping on the cliques and use that to estimate the sample probabilities. 


### Problem 3

(30) The following questions are based on reading and running the jupyter
notebook, `pymc3-variation-inference-neural-network.ipynb`,
by Thomas Wiecki, updated by Maxim Kochurov as provided in their blog
post. Run the notebook and then answer these questions.

####  Part a

Wieki says that an advantage to using Bayesian modeling with neural
network and deep learning is that "we could train the model specifically on samples it is most uncertain about." Explain how he finds
these samples in this example. Explain how you would implement
his suggestion (you do not have to actually implement this).

**Answer**

Thomas Wiecki calculates the standard deviation of the posterior predictive to get a sense for the uncertainty in his predictions. He also created a heapmap based on the standard deviation of each posterior prediction to get a sense of where the predictions surface is with high standard deviations.  

With the standard deviation calculated, I would separated the data with highest uncertainty out, for example top 25% of the data with highest standard deviation on posterior prediction. Then, I would build a new model on those 25% of the data. Once I have the new model, I will refit the entire dataset (100%) using the new model to get a new posterior prediction. 


#### Part b

Wieki also says that another advantage to Bayesian modeling with
neural network and deep learning is that \We also get uncertainty
estimates of our weights which could inform us about the stability
of the learned representations of the network." Discuss what the
uncertainty estimates for the weights found for the example in this
notebook imply.

**Answer**

Different layers of the neural network has different weights for the same predictor. We can view the uncertainty estimates for the weights same as the way we viewed the confidence interval for regression coefficients. If the uncertainty estimates for the weights covers the value of 0, that implies that such weights may not be useful in our model. 

#### Part c

Explain how the Gaussian priors help to regularize the weights in the
neural network.

**Answer**

Since we are using MAP estimation for the posterior distribution, we are trying to maximize the probability of the parameter given the data: 

$$P(W|x, y) = \frac{P(x, y | W) P(W)}{P(x ,y)} $$

where $W$ is the weights of predictor $x$, and $y$ is the response variable.

Since $P(x, y)$ is the normalizing constant, we can ignore that part. The maximum log likelihood then become 

$$log(P(W|x, y)) = log(P(x, y | W)) +  log(P(W))$$

If we assume the Gaussian priors, each element of the weights are drawn independently from a unit Gaussian, then 

$$log(P(W)) = log( \frac{1}{\sqrt{2\pi}} \prod_{ij} exp(- \frac{w_{ij}^2}{2} )) = -\frac{1}{2} \sum_{ij} w_{ij}^2 +  log( \frac{1}{\sqrt{2\pi}}) $$

Now the log likelihood becomes 

$$log(P(W|x, y)) = log(P(x, y | W)) - \frac{1}{2} \sum_{ij} w_{ij}^2 + c$$
where $c = log( \frac{1}{\sqrt{2\pi}})$, which is a constant.

Therefore, the regularitzation penalty $R(W) = \frac{1}{2} \sum_{ij} w_{ij}^2$.


Reference: https://stats.stackexchange.com/questions/229415/why-is-regularization-interpreted-as-a-gaussian-prior-on-my-weights

#### Part d

Why do we use a variational apprxomiation instead of sampling for estimating the posterior of the weights? 

**Answer**

Because the sampling method cannot scale to very large data sets and it can be very slow when working with high dimensional parameters or latent space. The posterior distribution may also be very complex to draw samples from.

#### Part e

Change the prior distributinos for all three sets of the neural net weights to Cauchy with location (alpha) = 0 and scale (beta) = 2. Rerun the remaining cells in the notebook and comment on any changes you see from this. 

**Answer**
- The Average Loss coming out of the ADVI model is higher.
- The toal prediction accuracy becomes better, changing from 95% to 96.4%
- The final Minibatch-ADVI model now have 3 weights that does not contain 0 in its corresponding distribution. Previously, there were only 2.


### Problem 4

(30) You are tracking the performance of a set of companies with the idea
that you might possibly buy stock in them. You decide to automate this process using HMM and you implement your first version for one company.
This company has three states that are hidden from investors: (1) in-trouble; (2) static; and (3) major growth potential. You have estimated
the transition probabilities between states as follows:

$$
\begin{bmatrix}
.6 & .3 & .1\\ 
.4 & .4 & .2\\ 
.1 & .4 & .5
\end{bmatrix}
$$

You have a text analysis system using Naive Bayes to process the quarterly
reports and assess their sentiment into one of three categories: (1) Fine;
(2) Good; and (3) Very good. Your estimates for the probabilities of these
sentiments given the state of the company are shown in the following
matrix (the sentiments are in the rows and the states of the company are
in the columns).

$$
\begin{bmatrix}
.45 & .4 & .15\\ 
.3 & .4 & .3\\ 
.2 & .5 & .3
\end{bmatrix}
$$

You have 3 quarterly reports with the assessments: Fine, Fine, Very Good
and your prior for the initial state is equally likely for each value. The
following questions use the HMM class in the jupyter notebook, HMM
Examples HW5 - Burglary and Investment with your additions
to it as indicated in the notebook.

In [6]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Nov 12 20:35:27 2020

@author: donaldbrown
"""

import numpy as np
import pandas as pd

class HMM:
    """Creates a class for Hidden Markov Models
    Input:
        Viz:      List of observed or visible states over time
        Trans_M:  Transition matrix for hidden states, H X H, H=len(Trans_M), no. of hidden variables
        Obs_M:    Observation matrix, H X V, V = no. of visible variables
        Pi:       List of initial state probabilities
    Methods:
        filter = The posterior probabilities for hidden states for each time period, T X H array
        smoother = The probabiliteis for the hidden states at each prevoius time period, T X H array
        viturbi = The most likely path of hidden states given the observed state, data frame, 1 X T
        predictor = The probabilities for next hidden state and the next observed state, 1 X H array """
    
    def __init__(self,Viz, Trans_M, Obs_M, Pi):
        # initialize variables
        # Hidden state transition matrix
        self.Trans_M = Trans_M
        # Visible or observates state probabilities given the hidden states
        self.Obs_M = Obs_M
        # No. of hidden states
        self.H = Trans_M.shape[0]
        # No. of observed states
        self.V = Obs_M.shape[0]
        # prior probabaiities for the hidden states
        self.Pi = Pi
        # List of observed states over time
        self.Viz = Viz

        
    def filter(self):
        
        T = len(self.Viz)
        
        # Obtain the joint probabilities of the hidden and observed states at time t
        self.alpha = np.zeros((T, self.H))
        self.alpha[0, :] = self.Pi * self.Obs_M[:,self.Viz[0]]
 
        for t in range(1, T):
            for j in range(self.H):
                self.alpha[t, j] = self.alpha[t - 1].dot(self.Trans_M[:, j]) * self.Obs_M[j, self.Viz[t]]
        
        ### Insert your code here to computer the posterior probabilities ###
        self.Post = np.zeros((T, self.H))
        
        for t in range(0, T):
            sum_of_row = sum(self.alpha[t,:])
            for j in range(self.H):
                self.Post[t, j] = self.alpha[t, j] / sum_of_row

        print("self.alpha")
        print(self.alpha)
        print("Posterior")
        print(self.Post)   
        return self.Post
      
    def smoother(self):

        T = len(self.Viz)
        self.beta = np.zeros((T, self.H))
 
        # setting beta(T) = 1
        self.beta[T - 1] = np.ones((self.H))
 
        # Loop backwards way from T-1 to 1
        # Due to python indexing the actual loop will be T-2 to 0
        for t in range(T - 2, -1, -1):
            for j in range(self.H):
                self.beta[t, j] = (self.beta[t + 1] * self.Obs_M[:, self.Viz[t + 1]]).dot(self.Trans_M[j, :])
                
        # Obtain the posterior probabilities of the hidden states given the observed states       
        
        ### Insert your code here to compute the posterior probabilities ###    
        self.Post_smoother = np.zeros((T, self.H))
        
        for t in range(0, T):
            sum_of_row = sum(self.alpha[t] * self.beta[t])
            
            for j in range(self.H):
                self.Post_smoother[t, j] = (self.alpha[t, j] * self.beta[t, j])  / sum_of_row      
        
        print("beta")
        print(self.beta)
        print("Posterior")
        print(self.Post_smoother)
 
        return self.Post_smoother
    
    
    def viturbi(self):
        T = len(self.Viz)
        
        # Obtain the joint probabity of the most likely path that ends in state j at time t
        delta = np.zeros((T, self.H))
        delta[0, :] = (self.Pi * self.Obs_M[:, Viz[0]])
 

        prev = np.zeros((T, self.H))
        prev[0,:] = np.repeat(None, 3)
 
        for t in range(1, T):
            for j in range(self.H):
                # The most likely state given our previoius state at t-1
                
                prob = delta[t - 1] * (self.Trans_M[:, j])
 
                #  The probability of the most probable state given the previous state and the observation at time t
                
                delta[t, j] = np.max(prob) * (self.Obs_M[j, Viz[t]])                
                
                # The most probable state given previous state 

                prev[t, j] = np.argmax(prob)
 
                
        # Path Array
        S = np.zeros(T)
 
        # Find the most probable last hidden state
        last_state = np.argmax(delta[T-1, :])
 
        S[T-1] = last_state
        
        # Find the most probable hidden states at the previous times
        # Walk backwords
        ### Insert your code here ###
        for t in range(T-1, 0, -1):
            S[t] = np.argmax(delta[t, :])
            
        # Change to states numbers in problem (i.e., +1)
        S = S+1
            
        S = S.reshape([1,3])
 
        # Path, S, as a dataframe 
        # Create a list of column names, Time  
        cols = list()
        for i in range(1,T+1):
            cols.append("Time "+(str(i)))
        Path = pd.DataFrame(S, columns = cols)
        print('delta')
        print(delta)
        print('Previous')
        print(prev)        
        print("Path")
        print(Path)
        return Path
 

    def predictor(self, steps = 1):
        T = len(self.Viz)
        # Hidden state prediction probabilities using filtering results (Post)
        ### Insert your code here ### dot product of filter post and transition matrix
        Pred_Hidden = Pred_Hidden = self.Post[T-1, :] @ self.Trans_M 
        print("Predicted Hidden State")
        print(Pred_Hidden)
        # Visible state prediction using the predicted hidden state probabilities
        ### Insert your code here ### dot product pred_hidden with observed matrix
        Pred_Visible = Pred_Visible = Pred_Hidden @ self.Obs_M
        print("Predicted Visible State")
        print(Pred_Visible)

In [7]:
# Transition matrix
TM = np.array([[.6,.3,.1],[.4,.4,.2],[.1,.4,.5]])  
# Observation matrix
OM = np.array([[.45,.4,.15],[.3,.4,.3],[.2,.5,.3]])
OM = OM.T
# # Prior probabilities of hidden states
p = [1/3, 1/3, 1/3]
# # Observed visible states
Viz = [0, 0, 2]

#### Part a 
In order to decide whether to invest, find the most likely current state
given the observed states.

In [8]:
# Use Filter to infer the present: get the current state
hmm1 = HMM(Viz, TM, OM, p)
hmm1.filter()

self.alpha
[[0.15       0.13333333 0.05      ]
 [0.06675    0.04733333 0.01      ]
 [0.01199667 0.02147917 0.0063425 ]]
Posterior
[[0.45       0.4        0.15      ]
 [0.53794493 0.38146407 0.080591  ]
 [0.301285   0.53942907 0.15928592]]


array([[0.45      , 0.4       , 0.15      ],
       [0.53794493, 0.38146407, 0.080591  ],
       [0.301285  , 0.53942907, 0.15928592]])

The most likely current state given the observed states is **Static**.

#### Part b

(b) Using smoothing to find the most likely state at each previous time
period (i.e., periods 1 and 2)..

In [9]:
hmm1.smoother()

beta
[[0.12735 0.1195  0.09565]
 [0.3     0.34    0.37   ]
 [1.      1.      1.     ]]
Posterior
[[0.47974133 0.40015068 0.12010799]
 [0.50290905 0.40416893 0.09292202]
 [0.301285   0.53942907 0.15928592]]


array([[0.47974133, 0.40015068, 0.12010799],
       [0.50290905, 0.40416893, 0.09292202],
       [0.301285  , 0.53942907, 0.15928592]])

The most likely state at time period 1 and 2 is **In Trouble, In Trouble**

#### Part c

Show the most likely path of performance through the hidden states
up to the current time.

In [10]:
hmm1.viturbi()

delta
[[0.15       0.13333333 0.05      ]
 [0.0405     0.02133333 0.004     ]
 [0.00486    0.006075   0.00128   ]]
Previous
[[nan nan nan]
 [ 0.  1.  1.]
 [ 0.  0.  1.]]
Path
   Time 1  Time 2  Time 3
0     1.0     1.0     2.0


,Time 1,Time 2,Time 3
0,1.0,1.0,2.0


The most likely path of perofrmance through the hidden states up to the current time is **In Trouble, In Trouble, Static**.

#### Part d

Find the most ikely hidden state and visible state for this company
in the next time period.

In [11]:
hmm1.predictor()

Predicted Hidden State
[0.41247122 0.3698715  0.21765728]
Predicted Visible State
[0.36620924 0.33698715 0.33272718]


- The most ikely hidden state for this company in the next time period is **In Trouble**.
- The most ikely visible state for this company in the next time period is **Fine**.